### Using the Bokeh to get the PV watts plot

First, i pull down the example from the Website

In [11]:
import requests
import pandas as pd

In [66]:
import numpy as np

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource

x = np.linspace(0, 10, 500)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(y_range=(-10, 10), plot_width=400, plot_height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase")
offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset")

callback = CustomJS(args=dict(source=source, amp=amp_slider, freq=freq_slider, phase=phase_slider, offset=offset_slider),
                    code="""
    const data = source.data;
    const A = amp.value;
    const k = freq.value;
    const phi = phase.value;
    const B = offset.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = B + A*Math.sin(k*x[i]+phi);
    }
    source.change.emit();
""")

amp_slider.js_on_change('value', callback)
freq_slider.js_on_change('value', callback)
phase_slider.js_on_change('value', callback)
offset_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(amp_slider, freq_slider, phase_slider, offset_slider),
)

output_file("slider.html", title="slider.py example")

show(layout)

OK, then i will do my data according this example.

In [12]:
#First call the PV Watts
nrel_long_tilt = []
tilts = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
for i in range(len(tilts)):
    list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 4, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": tilts[i], "azimuth": 180, "lat": 64.82, "lon": -147.87, "dataset": 'tmy2'}
    json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
    new_dataframe = pd.DataFrame(data = json_response['outputs'])
    nrel_long_tilt.append(new_dataframe)
print(nrel_long_tilt)

[    ac_monthly  poa_monthly  solrad_monthly  dc_monthly    ac_annual  \
0    18.987301     6.631680        0.213925   22.725245  3200.612793   
1    91.035156    27.050930        0.966105   98.086250  3200.612793   
2   272.492188    79.314285        2.558525  286.772400  3200.612793   
3   462.213074   135.295914        4.509864  483.805542  3200.612793   
4   519.547302   160.983749        5.193024  544.729736  3200.612793   
5   542.937012   171.665359        5.722178  569.626465  3200.612793   
6   511.257385   163.397552        5.270889  536.649658  3200.612793   
7   392.010071   122.991150        3.967456  412.263336  3200.612793   
8   233.482666    72.418793        2.413960  247.120529  3200.612793   
9   116.754417    35.631821        1.149414  125.749321  3200.612793   
10   34.762928    11.159619        0.371987   39.408035  3200.612793   
11    5.133535     2.533939        0.081740    7.657297  3200.612793   

    solrad_annual  capacity_factor  
0        2.701589        

In [65]:
ac_monthly = []
for i in range(len(nrel_long_tilt)):
    ac_monthly_each_month = []
    ac_monthly_each_month.append(nrel_long_tilt[i]['ac_monthly'])
    ac_monthly.append(ac_monthly_each_month)

date=[]
for i in np.arange(1,19):
    date.append(np.arange(1,13))

source = ColumnDataSource(data=dict(x=date, y=ac_monthly))

plot = figure(plot_width=400, plot_height=400)

plot.line(x='x', y='y', source=source, line_width=3, line_alpha=0.6)

tilt_slider = Slider(start=5, end=90, value=50, step=5, title="tilt")

callback = CustomJS(args=dict(source=source, tilt=tilt_slider,),
                    code="""
    const data = source.data;
    const tilt = tilt.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = y[i] ;
    }
    source.change.emit();
""")

tilt_slider.js_on_change('value', callback)


layout = row(
    plot,
    column(tilt_slider),
)

output_file("slider.html", title="slider.py example")

show(layout)

In [56]:
date=[]
for i in np.arange(1,19):
    date.append(np.arange(1,13))
    
ac_monthly = []
for i in range(len(nrel_long_tilt)):
    ac_monthly.append(nrel_long_tilt[i]['ac_monthly'])

df = pd.DataFrame({'month':"",'ac_monthly':ac_monthly,'tilt':""})

,month,ac_monthly,tilt
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[[18.987300872802734, 91.03515625, 272.4921875...",
1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[[26.884841918945312, 110.13851928710938, 304....",
2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[[35.10663604736328, 128.6448974609375, 333.81...",
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[[43.13510513305664, 146.09458923339844, 360.4...",
4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[[50.73208236694336, 162.25111389160156, 384.2...",
5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[[57.784568786621094, 176.99771118164062, 405....",
6,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[[64.24359130859375, 190.27540588378906, 423.2...",
7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[[70.08378601074219, 202.04953002929688, 438.2...",
8,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[[75.29331970214844, 212.30104064941406, 450.2...",
9,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[[79.8673095703125, 221.01853942871094, 459.00...",


In [64]:
import numpy as np

from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Select
from bokeh.plotting import figure

x = np.linspace(0, 10, 100)
foo = x**2
bar = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=foo, foo=foo, bar=bar))

plot = figure(plot_height=350)
plot.line(x='x', y='y', source=source)

select = Select(value='foo', options=['foo', 'bar'])
select.js_on_change('value', CustomJS(args=dict(source=source, select=select), code="""
    // make a shallow copy of the current data dict
    const new_data = Object.assign({}, source.data)

    // update the y column in the new data dict from the appropriate other column
    new_data.y = source.data[select.value]

    // set the new data on source, BokehJS will pick this up automatically
    source.data = new_data
"""))

show(column(plot, select))

In [5]:
import numpy as np
import pandas as pd

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource

month = [1,2,3,1,2,3,1,2,3,1,2,3]
tilts = [1,1,1,2,2,2,3,3,3,4,4,4]
data = [0.1,0.2,0.3,1,2,3,11,12,13,21,22,24]
df = pd.DataFrame({'month':month,'tilt':tilts,'data':data})

df_default = df[df['tilt']==2]

source = ColumnDataSource({
    'x': df_default.month.tolist(), 
    'y': df_default.data.tolist(), 
})

plot = figure(plot_width=400, plot_height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var tilt = slider.value;
    var x = data['x']
    var y = data['y']
    'x'        = df[df['tilt']==tilt].month.tolist();
    'y'        = df[df['tilt']==tilt].data;
    plot.line(x='x', y='y', source=source, line_width=3, 
line_alpha=0.6, 
         );
    source.change.emit();
""")

slider = Slider(start=1, end=4, step=1, value=1, title='tilt')
slider.js_on_change('value',callback)

layout = row(
    plot,
    column(slider)
)

output_file("slider.html", title="slider.py example")

show(layout)